In [1]:
from csd import CSD


In [2]:
example_csd = CSD()

[CSD] 0.0.1|INFO|2021-09-18 12:32:19]: Just a test that logger is working. Happy coding!


In [3]:
import strawberryfields as sf

In [4]:
displacement = 0.5

def single_layer():
    # Creates a single mode quantum "program".
    # https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.Program.html
    prog = sf.Program(1)

    with prog.context as q:
        # Phase space displacement gate.
        # https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.Dgate.html
        sf.ops.Dgate(0.0, 0.0) | q[0]

        # Measures whether a mode contain zero or nonzero photons.
        # https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.MeasureThreshold.html
        sf.ops.MeasureThreshold()       | q[0]

    return eng.run(prog)

In [5]:
eng = sf.Engine('gaussian')

In [6]:
result = single_layer()

/opt/conda/lib/python3.9/site-packages/thewalrus/samples.py:367: ComplexWarning: Casting complex values to real discards the imaginary part
/opt/conda/lib/python3.9/site-packages/thewalrus/samples.py:370: ComplexWarning: Casting complex values to real discards the imaginary part


In [7]:
print(result.samples)

[[0]]
